## Importer les librairies

In [94]:
import pandas as pd
import numpy as np

## Charger les datasets

In [95]:
df_quest = pd.read_csv('../Data/Anamnese.csv')
df_audio = pd.read_csv('../Data/Tonal.csv')
pd.set_option('display.max_columns', None)

C:\Users\thiev\AppData\Local\Temp\ipykernel_6920\2480570748.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_quest = pd.read_csv('../Data/Anamnese.csv')


---
## Nettoyer le dataframe anamnese

On supprime les colonnes inutiles

In [96]:
df_quest.drop(['EMPLOYEE_CODE', 'SHOP_CODE', 'QUESTION_DESCR', 'QUESTION_CODE'], axis=1, inplace=True)

On passe les questions en colonnes

In [97]:
df_quest = df_quest.pivot_table(index=["CUSTOMER_CODE", "QUESTIONNAIRE_DATE"], 
                          columns="QUESTION", 
                          values="ANSWER_VALUE",
                          aggfunc="first").reset_index()

On impute les valeurs manquantes de AVEZ_VOUS_DES_ACOUPHENES en fonction de INTENSITE_ACOUPHENES

In [98]:
#  Pour les lignes où ACOUPHENES est NaN mais NIVEAU_INTENSITE ou NIVEAU_GENE non-NaN, on met ACOUPHENES à 1
df_quest.loc[
    df_quest["AVEZ_VOUS_DES_ACOUPHENES"].isna() & 
    (df_quest["NIVEAU_INTENSITE_ACOUPHENE"].notna() | df_quest["NIVEAU_DE_GENE"].notna()), 
    "AVEZ_VOUS_DES_ACOUPHENES"
] = 1

# On supprime les lignes où AVEZ_VOUS_DES_ACOUPHENES est NaN ET NIVEAU_INTENSITE_ACOUPHENE est NaN
mask_drop = (
    df_quest['AVEZ_VOUS_DES_ACOUPHENES'].isna() &
    df_quest['NIVEAU_INTENSITE_ACOUPHENE'].isna() &
    df_quest['NIVEAU_DE_GENE'].isna()
)
df_quest = df_quest.loc[~mask_drop].copy()

# On supprime la colonne NIVEAU_INTENSITE_ACOUPHENE
df_quest = df_quest.drop(columns=['NIVEAU_INTENSITE_ACOUPHENE'])
df_quest = df_quest.drop(columns=['NIVEAU_DE_GENE'])

On affiche le dataframe anamnese nettoyé

In [99]:
display(df_quest)

QUESTION,CUSTOMER_CODE,QUESTIONNAIRE_DATE,ANTECEDENT_FAMILIAUX,AVEZ_VOUS_DES_ACOUPHENES,AVEZ_VOUS_DU_DIABETE,EXPOSITION_AU_BRUIT
0,00000000,2023-07-21T00:00:00.000Z,1.0,1.0,0.0,1.0
1,00000001,2023-06-07T00:00:00.000Z,0.0,1.0,0.0,1.0
3,00000035,2020-05-20T00:00:00.000Z,0.0,1.0,1.0,0.0
4,00000048,2024-07-30T00:00:00.000Z,0.0,1.0,NaN,1.0
6,00000089,2020-03-12T00:00:00.000Z,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...
353452,X0106065,2020-08-31T00:00:00.000Z,1.0,1.0,0.0,1.0
353453,X0106073,2023-03-16T00:00:00.000Z,1.0,0.0,0.0,0.0
353454,X0106081,2021-01-28T00:00:00.000Z,0.0,1.0,1.0,0.0
353455,X0106108,2021-05-18T00:00:00.000Z,0.0,0.0,0.0,1.0


---
## Nettoyer le dataframe tonal

On ne garde que les tests THR et UCL

In [100]:
df_audio = df_audio[df_audio['TYPE_TONALE'].isin(['Casque - Non appareille', 'UCL - Casque - Non appareille'])]

On supprime les colonnes inutiles

In [101]:
df_audio.drop(['PTA', 'CODAUPR', 'CODFILI', 'CONDITION', 'MASKED_TONAL', 'Unnamed: 0', 'SORTIE', 'TEST'], axis=1, inplace=True)
df_audio.drop(['FREQ_125', 'FREQ_750', 'FREQ_1500', 'FREQ_3000', 'FREQ_6000'], axis=1, inplace=True)

On renomme la colonne TYPE_TONALE en TEST et le type de test en THR ou UCL

In [102]:
df_audio.rename(columns={
    'TYPE_TONALE': 'TEST'
}, inplace=True)
df_audio['TEST'] = df_audio['TEST'].replace({'Casque - Non appareille': 'THR', 'UCL - Casque - Non appareille': 'UCL'})

On récupère tous les patients qui ont fait des tests UCL et THR pour les 2 oreilles et on crée des nouvelles colonnes pour les valeurs d'audiogrammes

In [103]:
df = df_audio.copy()
df['side_test'] = df['COTE'] + '_' + df['TEST']
expected = {'G_THR', 'G_UCL', 'D_THR', 'D_UCL'}
combos = df.groupby('CODANCL')['side_test'].apply(set)
full_patients = combos[combos.map(lambda s: expected.issubset(s))].index
meta = (
    df[['CODANCL','AGE','SESSO','DDCONT']]
    .drop_duplicates(subset=['CODANCL'])
    .set_index('CODANCL')
)
df_filt = df[
    df['CODANCL'].isin(full_patients) &
    df['side_test'].isin(expected)
]
df_unique = df_filt.drop_duplicates(subset=['CODANCL','COTE','TEST'])
freq_cols = ['FREQ_250', 'FREQ_500','FREQ_1000','FREQ_2000','FREQ_4000', 'FREQ_8000']
df_wide = (
    df_unique
    .set_index(['CODANCL','COTE','TEST'])[freq_cols]
    .unstack(['COTE','TEST'])
)
df_wide.columns = [
    f"{freq}_{side}_{test}"
    for freq, side, test in df_wide.columns
]
df_audio = (
    meta
    .join(df_wide, how='inner')
    .reset_index()
)

On passe toutes les colonnes de fréquence en int

In [104]:
for col in df_audio.columns:
    if col.startswith('FREQ'):
        df_audio[col] = df_audio[col].astype('float64')

On affiche le dataframe tonal nettoyé

In [105]:
display(df_audio)

,CODANCL,AGE,SESSO,DDCONT,FREQ_250_D_THR,FREQ_250_G_THR,FREQ_250_D_UCL,FREQ_250_G_UCL,FREQ_500_D_THR,FREQ_500_G_THR,FREQ_500_D_UCL,FREQ_500_G_UCL,FREQ_1000_D_THR,FREQ_1000_G_THR,FREQ_1000_D_UCL,FREQ_1000_G_UCL,FREQ_2000_D_THR,FREQ_2000_G_THR,FREQ_2000_D_UCL,FREQ_2000_G_UCL,FREQ_4000_D_THR,FREQ_4000_G_THR,FREQ_4000_D_UCL,FREQ_4000_G_UCL,FREQ_8000_D_THR,FREQ_8000_G_THR,FREQ_8000_D_UCL,FREQ_8000_G_UCL
0,62800702,67.0,U,2021-05-21T00:00:00.000Z,15.0,25.0,100.0,100.0,15.0,25.0,105.0,110.0,20.0,15.0,110.0,110.0,50.0,50.0,110.0,110.0,65.0,65.0,110.0,110.0,60.0,90.0,NaN,NaN
1,92603791,62.0,D,2023-03-24T00:00:00.000Z,45.0,40.0,100.0,100.0,45.0,40.0,105.0,105.0,40.0,35.0,105.0,105.0,30.0,40.0,105.0,110.0,40.0,40.0,105.0,110.0,55.0,60.0,NaN,NaN
2,25802062,61.0,D,2024-10-24T00:00:00.000Z,85.0,40.0,105.0,90.0,90.0,60.0,115.0,90.0,90.0,60.0,120.0,115.0,70.0,100.0,115.0,120.0,80.0,105.0,115.0,120.0,75.0,100.0,100.0,NaN
3,81967959,90.0,D,2024-01-16T00:00:00.000Z,55.0,80.0,90.0,NaN,55.0,90.0,95.0,115.0,60.0,85.0,100.0,110.0,75.0,90.0,105.0,115.0,80.0,110.0,105.0,0.0,85.0,105.0,110.0,NaN
4,19909777,85.0,D,2025-01-21T00:00:00.000Z,50.0,10.0,100.0,95.0,55.0,25.0,95.0,95.0,45.0,20.0,95.0,95.0,70.0,15.0,105.0,100.0,85.0,40.0,115.0,110.0,105.0,75.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216174,03000477,91.0,D,2022-04-01T00:00:00.000Z,35.0,35.0,90.0,100.0,40.0,45.0,90.0,95.0,50.0,55.0,90.0,90.0,60.0,70.0,95.0,90.0,65.0,70.0,95.0,95.0,85.0,60.0,100.0,105.0
216175,48816455,82.0,D,2021-02-25T00:00:00.000Z,25.0,20.0,95.0,90.0,25.0,20.0,90.0,85.0,25.0,20.0,100.0,95.0,20.0,20.0,100.0,100.0,50.0,50.0,105.0,95.0,50.0,55.0,95.0,95.0
216176,18500159,NaN,D,2024-01-24T00:00:00.000Z,35.0,30.0,100.0,95.0,40.0,20.0,100.0,95.0,50.0,25.0,105.0,95.0,55.0,30.0,110.0,95.0,90.0,30.0,110.0,95.0,95.0,55.0,NaN,NaN
216177,96705640,80.0,U,2025-01-28T00:00:00.000Z,30.0,25.0,105.0,105.0,45.0,45.0,115.0,115.0,45.0,45.0,120.0,120.0,50.0,55.0,120.0,120.0,50.0,70.0,120.0,120.0,85.0,90.0,105.0,105.0


---
## Fusionner les 2 jeux de données

In [106]:
df_final = df_quest.merge(df_audio, left_on="CUSTOMER_CODE", right_on="CODANCL", how="inner")

On garde les patients qui ont passé le test à moins de 30 jours du questionnaire

In [107]:
def filter_dates(df):
    df['QUESTIONNAIRE_DATE'] = pd.to_datetime(df['QUESTIONNAIRE_DATE'])
    df['DDCONT']             = pd.to_datetime(df['DDCONT'])

    mask = (df['DDCONT'] - df['QUESTIONNAIRE_DATE']).abs() <= pd.Timedelta(days=30)
    df = df.loc[mask]
    return df

df_final = filter_dates(df_final)

On enlève les colonnes inutiles

In [108]:
def drop_useless_columns(df):
    df.drop(columns=['QUESTIONNAIRE_DATE', 'DDCONT', 'CODANCL'], inplace=True)
    return df

df_final = drop_useless_columns(df_final)

On renomme les colonnes pour plus de clareté

In [109]:
def rename_columns(df):
    df.rename(columns={
        'SESSO': 'SEXE',
        'EXPOSITION_AU_BRUIT': 'EXPOSITION',
        'AVEZ_VOUS_DES_ACOUPHENES': 'ACOUPHENES',
        'AVEZ_VOUS_DU_DIABETE': 'DIABETE',
        'CUSTOMER_CODE': 'ID_CLIENT'
    }, inplace=True)
    df['SEXE'] = df['SEXE'].replace({'D': 'F', 'U': 'H'})
    df['SEXE'] = df['SEXE'].astype('category')
    return df

df_final = rename_columns(df_final)

On enlève les patients qui sont en double

In [110]:
df_final = df_final.drop_duplicates(subset='ID_CLIENT')

Pour les colonnes binaires, on remplace les floats 0 ou 1 par des booléens False ou True pour plus de clareté

In [111]:
colonnes = ['ANTECEDENT_FAMILIAUX', 'ACOUPHENES', 'DIABETE', 'EXPOSITION']
for col in colonnes:
    df_final = df_final[df_final[col].isin([0.0, 1.0]) | df_final[col].isna()]
    df_final[col] = df_final[col].replace({0.0: False, 1.0: True})
    df_final[col] = df_final[col].astype('boolean')

C:\Users\thiev\AppData\Local\Temp\ipykernel_6920\1893255091.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final[col] = df_final[col].replace({0.0: False, 1.0: True})


On enlève les valeurs aberrantes

In [112]:
df_final = df_final[(df_final['AGE'] >= 0) & (df_final['AGE'] <= 110)]

On reset la colonne index

In [113]:
df_final.reset_index(drop=True, inplace=True)

On affiche le dataframe

In [114]:
display(df_final)

,ID_CLIENT,ANTECEDENT_FAMILIAUX,ACOUPHENES,DIABETE,EXPOSITION,AGE,SEXE,FREQ_250_D_THR,FREQ_250_G_THR,FREQ_250_D_UCL,FREQ_250_G_UCL,FREQ_500_D_THR,FREQ_500_G_THR,FREQ_500_D_UCL,FREQ_500_G_UCL,FREQ_1000_D_THR,FREQ_1000_G_THR,FREQ_1000_D_UCL,FREQ_1000_G_UCL,FREQ_2000_D_THR,FREQ_2000_G_THR,FREQ_2000_D_UCL,FREQ_2000_G_UCL,FREQ_4000_D_THR,FREQ_4000_G_THR,FREQ_4000_D_UCL,FREQ_4000_G_UCL,FREQ_8000_D_THR,FREQ_8000_G_THR,FREQ_8000_D_UCL,FREQ_8000_G_UCL
0,00000001,False,True,False,True,55.0,H,20.0,30.0,100.0,100.0,25.0,40.0,105.0,105.0,25.0,40.0,105.0,105.0,35.0,55.0,110.0,110.0,60.0,65.0,110.0,110.0,NaN,NaN,NaN,NaN
1,00000105,False,True,False,False,77.0,F,20.0,35.0,100.0,100.0,35.0,30.0,110.0,105.0,30.0,30.0,115.0,105.0,35.0,35.0,110.0,110.0,60.0,65.0,115.0,120.0,60.0,60.0,NaN,NaN
2,00000162,False,True,False,False,63.0,F,25.0,30.0,85.0,85.0,50.0,45.0,90.0,95.0,80.0,80.0,105.0,100.0,80.0,85.0,105.0,105.0,105.0,100.0,120.0,115.0,105.0,105.0,NaN,NaN
3,00000345,False,True,False,True,65.0,F,25.0,15.0,95.0,90.0,20.0,10.0,100.0,90.0,5.0,10.0,100.0,95.0,10.0,15.0,100.0,95.0,15.0,10.0,90.0,100.0,40.0,35.0,90.0,85.0
4,00001120,False,True,False,True,0.0,F,15.0,10.0,83.0,88.0,20.0,15.0,95.0,0.0,15.0,15.0,87.0,94.0,20.0,20.0,89.0,87.0,30.0,35.0,90.0,94.0,90.0,95.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62270,A7406761,True,False,False,False,85.0,F,40.0,50.0,75.0,90.0,40.0,40.0,75.0,90.0,35.0,35.0,80.0,85.0,50.0,50.0,85.0,85.0,65.0,55.0,95.0,90.0,50.0,60.0,85.0,95.0
62271,A7406762,True,True,False,True,84.0,H,35.0,30.0,90.0,100.0,40.0,40.0,90.0,90.0,60.0,60.0,100.0,95.0,80.0,80.0,110.0,115.0,105.0,110.0,0.0,0.0,95.0,95.0,NaN,NaN
62272,A7406764,True,False,True,True,74.0,H,20.0,35.0,80.0,95.0,15.0,25.0,80.0,95.0,30.0,30.0,85.0,95.0,75.0,65.0,105.0,100.0,85.0,80.0,110.0,105.0,65.0,70.0,95.0,95.0
62273,A7406796,False,True,True,True,74.0,H,70.0,55.0,100.0,90.0,80.0,50.0,105.0,90.0,70.0,45.0,100.0,85.0,60.0,40.0,95.0,85.0,85.0,90.0,100.0,110.0,90.0,95.0,NaN,NaN


On affiche le nombre de valeurs manquantes dans chaque colonne

In [115]:
total = df_final.isna().sum()
percent = 100 * df_final.isna().mean().round(3)
dtypes = df_final.dtypes

missing_df = pd.DataFrame({
    'type': dtypes,
    'nb_missing': total,
    'pct_missing': percent
})
display(missing_df)

,type,nb_missing,pct_missing
ID_CLIENT,object,0,0.0
ANTECEDENT_FAMILIAUX,boolean,2655,4.3
ACOUPHENES,boolean,0,0.0
DIABETE,boolean,2534,4.1
EXPOSITION,boolean,1463,2.3
AGE,float64,0,0.0
SEXE,category,19,0.0
FREQ_250_D_THR,float64,133,0.2
FREQ_250_G_THR,float64,154,0.2
FREQ_250_D_UCL,float64,21576,34.6


On exporte le dataframe en csv

In [116]:
def export_to_csv(df, nom_fichier):
    df.to_csv(nom_fichier,
            sep=';',
            index=False,
            header=True,
            encoding='utf-8')

df_final = export_to_csv(df_final, '../Data/Cleaned_Dataframe.csv')